In [70]:
# initial setup
%run "../../../common/0_notebooks_base_setup.py"


/Users/csuarezgurruchaga/Desktop/Digital-House/CLASE_30/dsad_2021/common
default checking
Running command `conda list`... ok
jupyterlab=2.2.6 already installed
pandas=1.1.5 already installed
bokeh=2.2.3 already installed
seaborn=0.11.0 already installed
matplotlib=3.3.2 already installed
ipywidgets=7.5.1 already installed
pytest=6.2.1 already installed
chardet=4.0.0 already installed
psutil=5.7.2 already installed
scipy=1.5.2 already installed
statsmodels=0.12.1 already installed
scikit-learn=0.23.2 already installed
xlrd=2.0.1 already installed
Running command `conda install --yes nltk=3.5.0`... ok
Solving environment: ...working... done

# All requested packages already installed.


unidecode=1.1.1 already installed
pydotplus=2.0.2 already installed
pandas-datareader=0.9.0 already installed
flask=1.1.2 already installed


---

<img src='../../../common/logo_DH.png' align='left' width=35%/>

# <h1><center><ins>GRIDSEARCH</ins></center></h1>
<h1><center>Checkpoint:</center></h1>
<img src="img/01_gs.jpg" alt="Drawing" style="width: 300px;"/>

<a id="tabla_contenidos"></a> 
## Tabla de Contenidos

### <a href='#section_objetivos'>0. Objetivos de la Notebook</a>


### <a href='#section_repaso'>1. Lo que tenés que hacer es...</a>
- #### <a href='#section_reg_vs_clas'>1.1 ...cargar el dataset y preparar los datos para el modelo: `KNeighborsRegressor`</a>
- #### <a href='#1.2'>1.2 ...correr GridSearchCV:</a>
- #### <a href='#1.3'>1.3 ...ahora con RandomizedSearchCV:</a>

<a id="section_objetivos"></a> 
## 0. Objetivos de la Notebook

<div id="caja11" style="float:left;width: 100%;">
  <div style="float:left;width: 9%;"><img src="../../../common/icons/haciendo_foco.png" style="align:left"/> </div>
  <br>
  <div style="float:left;width: 85%;">
      <label>Poner en práctica lo aprendido en la notebook de práctica guiada.</label>
  <div style="float:left;width: 85%;">
      <label>Es importante que antes de la clase resuelvan esta notebook ya que es fundamental que sepan utilizar las herramientas que vimos en la práctica guiada para después trabajar el caso práctico en la clase.</label>        
</div>    
</div>

<a href='#tabla_contenidos'>Volver a TOC</a>

***

<a id="section_repaso"></a> 
## 1. Lo que tenés que hacer es...

<a id="section_reg_vs_clas"></a>
### 1.1 ...cargar el dataset y preparar los datos para el modelo [`KNeighborsRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html):

Vamos a volver a utilizar el dataset de la práctica guiada que tenía todos los jugadores de fútbol argentino que están en el juego fifa-18, y contaremos con las mismas variables, salvo que **puntaje_global** va a ser una variable numérica y no categorial: 
- **ID**: un número único que identifica al jugador en toda la base.
- **full_name**: nombre completo del jugador.
- **age**
- **club**: del jugador
- **height_cm**
- **weight_kg**
- **puntaje_global**: puntaje que identifica la habilidad general del jugador.
- **potential**: potencia física del jugador.
- **ritmo**: velocidad de aceleración del jugador.
- **disparos**: nivel de precisión y potencia de sus remates.
- **pases**: nivel de precisión en sus pases.
- **amagues**: nivel de habilidad para amagar a un rival.
- **defensa**: capacidad defensiva general del jugador.
- **físico**: estado físico del jugador (nos indicaría qué tan rápido se cansa)

El **objetivo** del dataset es utilizar distintas features vinculadas al jugador para poder predecir qué nivel global tiene.

Tal como hicimos en la práctica guiada, vamos a probar un modelo de vecinos cercanos pero en lugar de usarlo para clasificar, lo vamos a utilizar para una regresión: [`KNeighborsRegressor`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)). Y nuevamente, la idea es buscar cuáles es la combinación de hiperparámetros (`n_neighbors` y `weights`) que mejor predicen el nivel global del jugador. 


**CUIDADO:** revisá en la notebook de la **Práctica Guiada** todos los pasos previos que hicimos antes de instanciar y entrenar el modelo.

In [71]:
# Importá las librerías que vas a necesitar...
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [72]:
# Cargá en un dataframe el data set...
df=pd.read_csv('../Data/fifa_18_jugadores_argentinos_dos.csv',index_col='Unnamed: 0')
df.sample(5)

,ID,full_name,age,club,height_cm,weight_kg,puntaje_global,potential,ritmo,disparos,pases,amagues,defensa,físico
13849,233780,Alexis Maldonado,19,Banfield,187.0,84.0,61,75,58,28,33,37,61,72
8714,226545,Ezequiel Garré,35,Club Atlético Huracán,170.0,74.0,67,67,65,54,68,64,63,65
1456,213432,Facundo Ferreyra,26,Shakhtar Donetsk,187.0,76.0,76,79,72,76,58,72,26,68
5581,123603,Hugo Campagnaro,37,Pescara,181.0,85.0,70,70,46,39,52,52,70,74
2470,198347,Franco Jara,28,Pachuca,181.0,76.0,74,74,78,74,67,75,33,67


In [73]:
# Separamos las X y las Y (puntaje_global)...
cols=['age','height_cm','weight_kg','potential','ritmo','disparos','pases','amagues','defensa','físico']
X = df[cols]
y = df.puntaje_global

In [74]:
# Hacemos el train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 1)

In [75]:
# Utilizamos sklearn para estandarizar la matriz de Features
from sklearn.preprocessing import StandardScaler

esc = StandardScaler()
X_train = esc.fit_transform(X_train)
X_test = esc.transform(X_test)

<a href='#tabla_contenidos'>Volver a TOC</a>

<a id="1.2"></a>
### 1.2 ...correr GridSearchCV:

In [76]:
# instanciamos el modelo
from sklearn.neighbors import KNeighborsRegressor

knn_regressor = KNeighborsRegressor()

In [77]:
# armamos los parámetros para GridSearchCV
from sklearn.model_selection import StratifiedKFold

param_grid = {'weights':['uniform','distance'], 'n_neighbors':list(range(1, 31))}
folds = StratifiedKFold(n_splits=10, shuffle = True, random_state=19)

In [78]:
# importamos e instanciamos el GridSearchCV
from sklearn.model_selection import GridSearchCV

grid=GridSearchCV(knn_regressor, param_grid,cv=folds)

In [79]:
# entrenamos el modelo
grid.fit(X_train, y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=19, shuffle=True),
             estimator=KNeighborsRegressor(),
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30],
                         'weights': ['uniform', 'distance']})

In [86]:
# ¿cuál fue la mejor combinación de hiperparámetros encontrada por GridSearchCV? ¿cuál fue su score de validación?
print(grid.best_estimator_,  grid.best_score_)


KNeighborsRegressor(weights='distance') 0.8965201911416066


In [81]:
grid.best_params_

{'n_neighbors': 5, 'weights': 'distance'}

In [82]:
# ¿y cómo le fue al mejor modelo con los datos de test?
y_pred=grid.predict(X_test)

In [83]:
from sklearn.metrics import r2_score

r2_score(y_test, y_pred)

0.9166852521178354

<a href='#tabla_contenidos'>Volver a TOC</a>

<a id="1.3"></a>
### 1.3 ...ahora con RandomizedSearchCV::

In [114]:
# usando la misma grilla de parámetros, probá elegir los hiperparámetros con RandomizedSearchCV y evaluarlos

param_random = {'weights':['uniform','distance'], 'n_neighbors':list(range(1, 31))}
folds = StratifiedKFold(n_splits=10, shuffle = True, random_state=19)

In [115]:
from sklearn.model_selection import RandomizedSearchCV

random_search = RandomizedSearchCV(knn_regressor, param_random, n_iter=20)

In [116]:
random_search.fit(X_train, y_train)

RandomizedSearchCV(estimator=KNeighborsRegressor(), n_iter=20,
                   param_distributions={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8,
                                                        9, 10, 11, 12, 13, 14,
                                                        15, 16, 17, 18, 19, 20,
                                                        21, 22, 23, 24, 25, 26,
                                                        27, 28, 29, 30],
                                        'weights': ['uniform', 'distance']})

In [117]:
random_search.best_estimator_

KNeighborsRegressor(n_neighbors=4)

In [118]:
random_search.best_params_

{'weights': 'uniform', 'n_neighbors': 4}

In [119]:
y_pred_random = random_search.predict(X_test)

In [113]:
r2_score(y_test, y_pred_random)

0.9166852521178354

<a href='#tabla_contenidos'>Volver a TOC</a>